# Django 7 - Testing

### What are automated tests?

Tests are simple routines that check the operation of your code. Testing operates at different levels. Some tests might apply to a tiny detail (does a particular model method return values as expected?) while others examine the overall operation of the software (does a sequence of user inputs on the site produce the desired result?). That’s no different from the kind of testing you did earlier in Django 3 Tutorial, using the shell to examine the behavior of a method, or running the application and entering data to check how it behaves.

What’s different in automated tests is that the testing work is done for you by the system. You programmatically create a set of tests once, and then as you make changes to your app, you can automatically check that your code still works as you originally intended using your automated testing scripts, which liberates you from having to perform time consuming manual testing.

### Why you need to create tests?

You may feel that ‘checking that my web application seems to work’ will be a satisfactory test in a work environment. However, when working with sophisticated application ‘checking that my web application seems to work’ is not good enough. In a complex application you might have dozens of complex/obscure interactions between components and usually several developers working on the application. A change in any of those components could have unexpected/unintended/unknown consequences on the application’s behavior. Checking that it still ‘seems to work’ could mean running through your code’s functionality with twenty different variations of your test data just to make sure you haven’t broken something - not a good use of your time.

That’s especially true when automated tests could do this for you in seconds. If something’s gone wrong, tests will also assist in identifying the code that’s causing the unexpected behavior. Sometimes it may seem a chore to do the unglamorous and unexciting business of writing tests, particularly when you know your code is working properly. However, the task of writing tests is more fulfilling than spending hours testing your application manually or trying to identify the cause of a newly-introduced problem. 

Additionally, when working in complex applications maintained by teams, tests guarantee that colleagues don’t inadvertently break your code (and that you don’t break theirs without knowing). If you want to make a living as a programmer, you must be good at writing tests!

### Basic testing strategies

There are many ways to approach writing tests.

Some programmers follow a discipline called “*test-driven development*”. This means that the programmer actually writes tests before he writes his code. More often, a newcomer to testing will create some code and later decide that it should have some tests. Perhaps it would have been better to write some tests earlier, but it’s never too late to get started. Sometimes you're working on code you know is reliable. In such a case, it’s fruitful to write your first test the next time you make a change, either when you add a new feature or fix a bug.

### Introducing a bug in or Blog application

Let's say that for whatever reason you want to programmatically check whether a post was published recently or not. We create a new method in the `Post` model (located at `mysite/blog/models.py`) to check whether a certain post was published within the last 24 hours:

In [3]:
import datetime
def was_published_recently(self):
    return self.published_date >= timezone.now() - datetime.timedelta(days=1)

Right now, there’s a bug in our blog application: the `Post.was_published_recently()` method returns `True` if the post was published within the last day (which is correct) but also if the post’s `pub_date` field is in the future (which certainly isn’t correct).

To check if the bug really exists, using the Admin create a Post whose date lies in the future and check the method using the Django shell:

In [ ]:
>>> import datetime
>>> from django.utils import timezone
>>> from blog.models import Post
>>> # create a Post instance with pub_date 30 days in the future
>>> future_post = Post(published_date=timezone.now() + datetime.timedelta(days=30))
>>> future_post.was_published_recently()

You will get a result of True from running the previous commands in the shell.  So you want to fix that bug, but you also want to have a test that ensures that this specific bug is not accidentally introduced in the future by some developer that modifies your data model or some other part of your blog application.

You can exit the Django shell by typing:

In [ ]:
>>> exit()

### Create a test to expose the bug

What we’ve just done manually in the shell to test for the problem is exactly what we can automate in a test, so let’s turn that code into an automated test.

A conventional place for an application’s tests is in the application’s `tests.py` file. Django testing system will automatically find tests in any file whose name begins with `test`.

Place the following code in `mysite/blog/tests.py`:

In [ ]:
import datetime
from django.utils import timezone
from django.test import TestCase
from .models import Post


class PostMethodTests(TestCase):
    def test_was_published_recently_with_future_post(self):
        """
        method was_published_recently() in Post model should return False for questions whose
        pub_date is in the future.
        """
        time = timezone.now() + datetime.timedelta(days=30)
        future_question = Post(published_date=time)
        self.assertEqual(future_question.was_published_recently(), False)

What we have done here is to create a `django.test.TestCase` named `PostMethodTests` subclass with a method that creates a `Post` instance with a `pub_date` attribute set to the future. We then check the output of the `was_published_recently()` method of the `Post` class - which ought to be False.

### Running tests

In the terminal, we can run our test:

and you’ll see something like:

What happened was this:

- `python manage.py test blog` looked for tests in the `blog` application
- it found a subclass of the `django.test.TestCase` class
- it created a special database for the purpose of testing
- it looked for test methods - ones whose names begin with test, such as our test method `test_was_published_recently_with_future_post`
- inside `test_was_published_recently_with_future_post` it created a `Post` instance whose `published_date` attribute is 30 days into the future
- using the `assertEqual()` method, it discovered that the `Post` class method `was_published_recently()` returns `True` for this particular test, though we wanted it to return False
- So basically our automatic testing unit has discovered a bug. The `python manage.py test blog` command informs us which specific test failed and even the line on which the failure occurred.

### Fixing the bug

Finally, we will amend the `Post` method `was_published_recently` of the Post model in `mysite/blog/models.py`, so that it will only return `True` if the `published_date` attribute of a Post instance is also in the past:

In [ ]:
def was_published_recently(self):
    now = timezone.now()
    return now - datetime.timedelta(days=1) <= self.published_date <= now

we can run our test again:

Notice that this time out automated test didn't raise any AssertionError. Congratulations! you successfully implemented your first test in Django. Let's recap what we have done:

After identifying a bug in our blog application, we wrote a test that exposes it in order to prevent this bug to emerge again in the future. Then, we corrected the bug in the code so our test passes. Many other things might go wrong with our application in the future, but we can be sure that we won’t inadvertently reintroduce this bug, because simply running the test will warn us about it immediately. We can consider this little portion of the application pinned down safely forever.

### Deploying our progress

Let’s upload today’s practical advances to our production server. Once again, from your git console type:

Then, log back in to https://www.pythonanywhere.com/ and go to your Bash console (or start a new one), and run:

Finally, hop on over to the Web tab and hit <font color="green">**Reload** </font> on your web app. Your update should be live!

---
<div align="right"><font size="1" >Partially adapted from https://tutorial.djangogirls.org/en/</font></div>